In [33]:
import pandas as pd
import os
import glob
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import statsmodels.api as sm
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor


# 1. Correct File Loading
medal_counts = pd.read_csv('/Users/chris/MCM_2025_C/Data/summerOly_medal_counts_with_codes.csv')
programs = pd.read_csv('/Users/chris/MCM_2025_C/Data/summerOly_programs.csv')
hosts = pd.read_csv('/Users/chris/MCM_2025_C/Data/summerOly_hosts_with_codes.csv')  # Corrected

# 2. Load Athlete Counts Data
pattern = os.path.join('/Users/chris/MCM_2025_C/Data/athlete_probabilities_by_year', '*.csv')
athlete_files = glob.glob(pattern, recursive=True)
athlete_dfs = []

for file in athlete_files:
    try:
        df = pd.read_csv(file)
        # Standardize column names
        df.rename(columns={
            'bronze': 'Bronze',
            'silver': 'Silver',
            'gold': 'Gold',
            'total_athletes': 'Total_Athletes',
            'year': 'Year'
        }, inplace=True)
        # Extract Year from filename if 'Year' column is absent
        if 'Year' not in df.columns:
            year_str = os.path.splitext(os.path.basename(file))[0].split('_')[-1]
            df['Year'] = int(year_str)
        athlete_dfs.append(df)
    except Exception as e:
        print(f"Error reading {file}: {e}")


### Process and Aggregate the athlete_probabilities_by_year DataFrame

In [34]:

# Concatenate all athlete DataFrames
athlete_counts = pd.concat(athlete_dfs, ignore_index=True)

# 3. Aggregate Athlete Counts
if 'Total_Athletes' in athlete_counts.columns:
    athlete_counts_agg = athlete_counts.groupby(['Year', 'Country Code']).agg({
        'Total_Athletes': 'sum'
    }).reset_index()
else:
    # If 'Total_Athletes' is not present, count the number of athletes
    athlete_counts_agg = athlete_counts.groupby(['Year', 'Country Code']).size().reset_index(name='Total_Athletes')


# 4. Standardize Country Codes Across DataFrames
def standardize_CountryCode(df, column='Country Code'):
    df[column] = df[column].str.upper().str.strip()
    return df

medal_counts = standardize_CountryCode(medal_counts, 'Country Code')
hosts = standardize_CountryCode(hosts, 'Country Code')        # Assuming hosts has 'Country Code'
athlete_counts_agg = standardize_CountryCode(athlete_counts_agg, 'Country Code')


### Process and Aggregate the programs DataFrame


In [35]:
# 2. Reshape 'programs' DataFrame from Wide to Long Format
# Identify year columns (assuming they are all numeric)
year_columns = [col for col in programs.columns if col.isdigit()]

# Melt the DataFrame
programs_long = programs.melt(
    id_vars=['Sport', 'Discipline', 'Code', 'Sports Governing Body'],
    value_vars=year_columns,
    var_name='Year',
    value_name='Event_Count'
)

# Convert 'Year' to integer
programs_long['Year'] = programs_long['Year'].astype(int)

# Preview the reshaped DataFrame
print("Reshaped Programs DataFrame (Long Format):")
print(programs_long.head())

Reshaped Programs DataFrame (Long Format):
      Sport         Discipline Code Sports Governing Body  Year  Event_Count
0  Aquatics  Artistic Swimming  SWA        World Aquatics  1896          0.0
1  Aquatics             Diving  DIV        World Aquatics  1896          0.0
2  Aquatics  Marathon Swimming  OWS        World Aquatics  1896          0.0
3  Aquatics           Swimming  SWM        World Aquatics  1896          4.0
4  Aquatics         Water Polo  WPO        World Aquatics  1896          0.0


In [36]:
# 3. Aggregate Event Counts and Number of Sports per Year
event_agg = programs_long.groupby('Year').agg(
    Total_Events=('Event_Count', 'sum'),
    Number_of_Sports=('Sport', 'nunique')
).reset_index()

# Preview the aggregated data
print("Aggregated Event Data per Year:")
print(event_agg.head())

Aggregated Event Data per Year:
   Year  Total_Events  Number_of_Sports
0  1896         107.0                51
1  1900         236.0                51
2  1904         224.0                51
3  1906         176.0                51
4  1908         267.0                51


## Merge All DataFrames into merged_df

In [37]:
if not medal_counts.empty and not athlete_counts_agg.empty:
    merged_df = pd.merge(
        medal_counts,
        athlete_counts_agg,
        on=['Year', 'Country Code'],
        how='left'
    )
    print("\nMerged Medal Counts with Athlete Counts successfully.")
else:
    merged_df = medal_counts.copy()
    merged_df['Total_Athletes'] = 0
    print("\nAthlete Counts Aggregated DataFrame is empty. 'Total_Athletes' set to 0 in merged_df.")



Merged Medal Counts with Athlete Counts successfully.


### Handle Missing Total_Athletes

In [38]:
if 'Total_Athletes' in merged_df.columns:
    merged_df['Total_Athletes'] = merged_df['Total_Athletes'].fillna(0).astype(int)
    print("'Total_Athletes' missing values filled with 0.")
else:
    merged_df['Total_Athletes'] = 0
    print("'Total_Athletes' column not found. Created and set to 0.")


'Total_Athletes' missing values filled with 0.


### Merge with hosts DataFrame to Set Is_Host Indicator

In [39]:
if not hosts.empty:
    # Assuming 'hosts' DataFrame has 'Year' and 'Country Code' indicating the host country each year
    host_info = set(hosts[['Year', 'Country Code']].drop_duplicates().itertuples(index=False, name=None))
    
    # Create 'Is_Host' column
    merged_df['Is_Host'] = merged_df.apply(
        lambda row: 1 if (row['Year'], row['Country Code']) in host_info else 0,
        axis=1
    )
    print("'Is_Host' indicator set successfully.")
else:
    merged_df['Is_Host'] = 0
    print("Hosts DataFrame is empty. 'Is_Host' set to 0.")


'Is_Host' indicator set successfully.


### Merge Aggregated Data

In [40]:
if not event_agg.empty:
    merged_df = pd.merge(
        merged_df,
        event_agg,
        on='Year',
        how='left'
    )
    # Fill missing values with 0
    merged_df['Total_Events'] = merged_df['Total_Events'].fillna(0).astype(int)
    merged_df['Number_of_Sports'] = merged_df['Number_of_Sports'].fillna(0).astype(int)
    print("Merged with Aggregated Event Data successfully.")
else:
    merged_df['Total_Events'] = 0
    merged_df['Number_of_Sports'] = 0
    print("Aggregated Event Data is empty. 'Total_Events' and 'Number_of_Sports' set to 0.")


# merged_df['Athletes_per_Event'] = merged_df['Total_Athletes'] / merged_df['Total_Events'].replace(0, 1)  # Avoid division by zero
# Check for unique Country Code
unique_countries = merged_df['Country Code'].nunique()
total_rows = merged_df.shape[0]
print(f"Unique countries: {unique_countries}")
print(f"Total rows in merged_df: {total_rows}")

if unique_countries != total_rows:
    print("Warning: There are duplicate Country Code entries. Aggregating data to ensure one row per country.")
    # Aggregate data (e.g., sum of features) to have one row per country
    merged_df = merged_df.groupby('Country Code').agg({
        'Total': 'sum',
        'Total_Athletes': 'sum',
        'Is_Host': 'max',  # Assuming Is_Host is binary (0 or 1)
        'Number_of_Sports': 'sum',
        'Total_Events': 'sum',
        # Add other relevant features as needed
    }).reset_index()
    print("Data aggregated to have one row per country.")
else:
    print("All Country Code entries are unique.")


print("\nFinal Merged DataFrame:")
print(merged_df.head())


Merged with Aggregated Event Data successfully.
Unique countries: 152
Total rows in merged_df: 1419
Data aggregated to have one row per country.

Final Merged DataFrame:
  Country Code  Total  Total_Athletes  Is_Host  Number_of_Sports  Total_Events
0          AFG      2              48        0               102          1343
1          AHO      1              32        0                51           528
2          ALB      2              26        0                51           738
3          ALG     20             656        0               408          5099
4          ANZ     12              40        0               102           503


### Define Variables for Modeling


In [41]:
# Define the dependent variable
dependent_var = 'Total'  # This is the total medal count

# Verify if 'Total' exists in merged_df
if dependent_var not in merged_df.columns:
    print(f"Error: Dependent variable '{dependent_var}' not found in merged_df.")
    # Optionally, inspect available columns
    print("Available columns:", merged_df.columns.tolist())
else:
    print(f"\nDependent variable set to '{dependent_var}'.")



Dependent variable set to 'Total'.


In [42]:
# Define the list of independent variables
independent_vars = ['Total_Athletes', 'Is_Host', 'Number_of_Sports', 'Total_Events']

# Verify if all independent variables exist in merged_df
missing_vars = [var for var in independent_vars if var not in merged_df.columns]
if missing_vars:
    print(f"Warning: The following independent variables are missing in merged_df: {missing_vars}")
    # Handle missing variables, e.g., create them with default values
    for var in missing_vars:
        merged_df[var] = 0
    print(f"Missing independent variables {missing_vars} created with default value 0.")
else:
    print("All independent variables are present in merged_df.")


All independent variables are present in merged_df.


In [43]:
# Define the output path
output_path = '/Users/chris/MCM_2025_C/Data/merged_data.csv'

# Save merged_df to CSV
try:
    merged_df.to_csv(output_path, index=False)
    print(f"\nMerged DataFrame saved successfully to {output_path}.")
except Exception as e:
    print(f"Error saving merged DataFrame to CSV: {e}")



Merged DataFrame saved successfully to /Users/chris/MCM_2025_C/Data/merged_data.csv.


## Split the Data into Training and Testing Sets

In [44]:
X = merged_df[independent_vars]
y = merged_df[dependent_var]

# Check for missing values in X and y
print("\nMissing values in X:")
print(X.isnull().sum())

print("\nMissing values in y:")
print(y.isnull().sum())

# Handle missing values if any
if X.isnull().values.any():
    X = X.fillna(X.median())
    print("Missing values in X filled with median.")

if y.isnull().values.any():
    y = y.fillna(0)
    print("Missing values in y filled with 0.")

# Split the data (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print("\nData split into training and testing sets successfully.")



Missing values in X:
Total_Athletes      0
Is_Host             0
Number_of_Sports    0
Total_Events        0
dtype: int64

Missing values in y:
0

Data split into training and testing sets successfully.


Feature Scaling

In [45]:
# 10. Feature Scaling (If Needed)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Feature scaling applied successfully.")


Feature scaling applied successfully.


## Predicitons and Model Evaluation

In [46]:

# # a. Poisson Regression
# X_train_sm = sm.add_constant(X_train_scaled)
# X_test_sm = sm.add_constant(X_test_scaled)

# poisson_model = sm.GLM(y_train, X_train_sm, family=sm.families.Poisson()).fit()
# print("\nPoisson Regression Model Summary:")
# print(poisson_model.summary())

# y_pred = poisson_model.predict(X_test_sm)

# mae = mean_absolute_error(y_test, y_pred)
# rmse = mean_squared_error(y_test, y_pred, squared=False)
# print(f"\nPoisson Regression - MAE: {mae:.2f}, RMSE: {rmse:.2f}")

# # b. Negative Binomial Regression
# nb_model = sm.GLM(y_train, X_train_sm, family=sm.families.NegativeBinomial()).fit()
# print("\nNegative Binomial Regression Model Summary:")
# print(nb_model.summary())

# y_pred_nb = nb_model.predict(X_test_sm)

# mae_nb = mean_absolute_error(y_test, y_pred_nb)
# rmse_nb = mean_squared_error(y_test, y_pred_nb, squared=False)
# print(f"\nNegative Binomial Regression - MAE: {mae_nb:.2f}, RMSE: {rmse_nb:.2f}")

# # c. Random Forest Regressor
# rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
# rf_model.fit(X_train, y_train)
# print("\nRandom Forest Regressor trained successfully.")

# y_pred_rf = rf_model.predict(X_test)

# mae_rf = mean_absolute_error(y_test, y_pred_rf)
# rmse_rf = mean_squared_error(y_test, y_pred_rf, squared=False)
# r2_rf = r2_score(y_test, y_pred_rf)
# print(f"\nRandom Forest Regressor - MAE: {mae_rf:.2f}, RMSE: {rmse_rf:.2f}, R²: {r2_rf:.2f}")

# Feature Scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Add a constant for the intercept
X_train_sm = sm.add_constant(X_train_scaled)
X_test_sm = sm.add_constant(X_test_scaled)

# Fit Poisson Regression
poisson_model = sm.GLM(y_train, X_train_sm, family=sm.families.Poisson()).fit()
print("\nPoisson Regression Model Summary:")
print(poisson_model.summary())



Poisson Regression Model Summary:
                 Generalized Linear Model Regression Results                  
Dep. Variable:                  Total   No. Observations:                  121
Model:                            GLM   Df Residuals:                      116
Model Family:                 Poisson   Df Model:                            4
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -6759.5
Date:                Mon, 27 Jan 2025   Deviance:                       12942.
Time:                        16:47:05   Pearson chi2:                 1.96e+04
No. Iterations:                     6   Pseudo R-squ. (CS):              1.000
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.

Simulate

In [50]:
import plotly.express as px
num_simulations = 1000
np.random.seed(42)  # For reproducibility

simulation_results = []
for sim in range(num_simulations):
    expected_counts = poisson_model.predict(X_train_sm)
    simulated_medals = np.random.poisson(expected_counts)
    simulation_results.append(simulated_medals)

print("\nSimulations completed successfully.")

# ---------------------------- #
# 17. Create Simulation DataFrame with Unique Country_Code Columns
# ---------------------------- #
# Extract unique Country_Code values from training set
train_cc_unique = train_cc.unique()
num_unique_train_countries = len(train_cc_unique)
print(f"\nNumber of unique training countries: {num_unique_train_countries}")

# Check if each simulation has the correct number of columns
if simulation_results and len(simulation_results[0]) == num_unique_train_countries:
    simulation_df = pd.DataFrame(simulation_results, columns=train_cc_unique)
    print("Simulation DataFrame created successfully with correct column alignment.")
else:
    print("Error: Mismatch between number of simulation columns and unique training countries.")
    print(f"Expected columns: {num_unique_train_countries}, Actual columns: {len(simulation_results[0]) if simulation_results else 0}")
    # Handle mismatch (e.g., adjust simulation_results or train_cc_unique)
    # For now, exiting to prevent further errors
    import sys
    sys.exit("Exiting due to column mismatch.")

# ---------------------------- #
# 18. Calculate Prediction Intervals
# ---------------------------- #
prediction_intervals = simulation_df.quantile([0.025, 0.5, 0.975]).T
prediction_intervals.columns = ['Lower_95%', 'Median', 'Upper_95%']
print("\nPrediction intervals calculated successfully.")
print(prediction_intervals.head())

# ---------------------------- #
# 19. Aggregate y_train by Country_Code
# ---------------------------- #
y_train_grouped = y_train.groupby(train_cc).sum()
print("\nAggregated y_train by Country_Code:")
print(y_train_grouped.head())

# ---------------------------- #
# 20. Align Prediction Intervals with Aggregated Data
# ---------------------------- #
# Ensure that train_cc_unique matches y_train_grouped.index
train_cc_unique_ordered = y_train_grouped.index.tolist()

# Reindex prediction_intervals to match train_cc_unique_ordered
prediction_intervals_sorted = prediction_intervals.reindex(train_cc_unique_ordered)

# Check for any NaNs introduced by reindexing
if prediction_intervals_sorted.isnull().values.any():
    print("\nWarning: Some prediction intervals could not be aligned with the aggregated Country_Code.")
    # Fill NaNs with 0 or appropriate default values
    prediction_intervals_sorted = prediction_intervals_sorted.fillna(0)
    print("Filled NaNs in prediction intervals with 0.")

print("\nPrediction Intervals after Sorting:")
print(prediction_intervals_sorted.head())

# ---------------------------- #
# 21. Create Final DataFrame with Prediction Intervals
# ---------------------------- #
final_with_intervals = pd.DataFrame({
    'Country_Code': train_cc_unique_ordered,
    'Total': y_train_grouped.values,
    'Lower_95%': prediction_intervals_sorted['Lower_95%'].values,
    'Median': prediction_intervals_sorted['Median'].values,
    'Upper_95%': prediction_intervals_sorted['Upper_95%'].values
})
print("\nFinal DataFrame with Prediction Intervals:")
print(final_with_intervals.head())

# ---------------------------- #
# 22. Save Final DataFrame to CSV
# ---------------------------- #
simulation_output_path = '/Users/chris/MCM_2025_C/Datamedal_count_simulation_2028.csv'
final_with_intervals.to_csv(simulation_output_path, index=False)
print(f"\nSimulation results saved successfully to {simulation_output_path}.")



Simulations completed successfully.

Number of unique training countries: 121
Simulation DataFrame created successfully with correct column alignment.

Prediction intervals calculated successfully.
     Lower_95%  Median  Upper_95%
CRC     24.000    35.0     46.025
CAN    713.000   768.0    830.025
GBR   1134.975  1204.5   1270.000
KAZ     28.000    40.0     52.000
BAR     21.000    32.0     43.000

Aggregated y_train by Country_Code:
Country Code
AHO      5
ALB      3
ALG      4
ANZ      1
ARG    981
Name: Total, dtype: int64

Prediction Intervals after Sorting:
     Lower_95%  Median  Upper_95%
AHO       21.0    32.0     44.000
ALB       21.0    31.0     43.000
ALG       27.0    40.0     52.025
ANZ       24.0    35.0     48.000
ARG       74.0    92.0    111.025

Final DataFrame with Prediction Intervals:
  Country_Code  Total  Lower_95%  Median  Upper_95%
0          AHO      5       21.0    32.0     44.000
1          ALB      3       21.0    31.0     43.000
2          ALG      4    